# Graph Store

## Overview

This notebook demonstrates how to store and query property graphs using Semantica's graph store modules. You'll learn to use `GraphStore` with multiple backends including Neo4j, KuzuDB, and FalkorDB.

### Learning Objectives

- Use `GraphStore` to store nodes and relationships
- Execute Cypher queries for graph retrieval
- Use graph analytics (shortest path, neighbors)
- Compare different graph database backends

---

## Prerequisites

Install the required graph database client:

```bash
# For Neo4j
pip install neo4j

# For KuzuDB (embedded - no server required)
pip install kuzu

# For FalkorDB
pip install falkordb
# And run: docker run -p 6379:6379 -p 3000:3000 -it --rm falkordb/falkordb
```


## Step 1: Initialize Graph Store

Create a graph store with your preferred backend.


In [ ]:
from semantica.graph_store import GraphStore

# Option 1: Neo4j (requires Neo4j server)
# store = GraphStore(
#     backend="neo4j",
#     uri="bolt://localhost:7687",
#     user="neo4j",
#     password="password"
# )

# Option 2: KuzuDB (embedded - no server required)
store = GraphStore(
    backend="kuzu",
    database_path="./demo_graph_db"
)

# Option 3: FalkorDB (requires Redis/FalkorDB server)
# store = GraphStore(
#     backend="falkordb",
#     host="localhost",
#     port=6379,
#     graph_name="demo_graph"
# )

# Connect to the database
store.connect()
print("Connected to graph store!")


## Step 2: Create Nodes

Create nodes with labels and properties.


In [ ]:
# Create individual nodes
apple = store.create_node(
    labels=["Company"],
    properties={"name": "Apple Inc.", "founded": 1976, "industry": "Technology"}
)
print(f"Created company node: {apple}")

tim_cook = store.create_node(
    labels=["Person"],
    properties={"name": "Tim Cook", "title": "CEO", "age": 63}
)
print(f"Created person node: {tim_cook}")

cupertino = store.create_node(
    labels=["Location"],
    properties={"name": "Cupertino", "state": "California", "country": "USA"}
)
print(f"Created location node: {cupertino}")


In [ ]:
# Create multiple nodes in batch
other_companies = store.create_nodes([
    {"labels": ["Company"], "properties": {"name": "Microsoft", "founded": 1975}},
    {"labels": ["Company"], "properties": {"name": "Google", "founded": 1998}},
    {"labels": ["Company"], "properties": {"name": "Amazon", "founded": 1994}},
])
print(f"Created {len(other_companies)} company nodes in batch")


## Step 3: Create Relationships

Create relationships between nodes.


In [ ]:
# Create relationships
ceo_rel = store.create_relationship(
    start_node_id=tim_cook["id"],
    end_node_id=apple["id"],
    rel_type="CEO_OF",
    properties={"since": 2011}
)
print(f"Created CEO relationship: {ceo_rel}")

location_rel = store.create_relationship(
    start_node_id=apple["id"],
    end_node_id=cupertino["id"],
    rel_type="HEADQUARTERED_IN",
    properties={"since": 1977}
)
print(f"Created location relationship: {location_rel}")


## Step 4: Query Nodes and Relationships

Retrieve nodes and relationships from the graph.


In [ ]:
# Get all Company nodes
companies = store.get_nodes(labels=["Company"], limit=10)
print(f"Found {len(companies)} companies:")
for company in companies:
    print(f"  - {company.get('properties', {}).get('name', 'Unknown')}")


In [ ]:
# Get relationships for a node
relationships = store.get_relationships(node_id=apple["id"], direction="both")
print(f"Found {len(relationships)} relationships for Apple:")
for rel in relationships:
    print(f"  - Type: {rel.get('type')}, Properties: {rel.get('properties')}")


## Step 5: Execute Cypher Queries

Use Cypher queries for complex graph operations.


In [ ]:
# Execute a Cypher query
results = store.execute_query("""
    MATCH (p:Person)-[r:CEO_OF]->(c:Company)
    RETURN p.name as person, c.name as company, r.since as since
""")

print("CEO relationships:")
for record in results.get("records", []):
    print(f"  {record}")


In [ ]:
# Parameterized query
results = store.execute_query(
    "MATCH (c:Company) WHERE c.founded > $year RETURN c.name, c.founded",
    parameters={"year": 1990}
)

print("Companies founded after 1990:")
for record in results.get("records", []):
    print(f"  {record}")


## Step 6: Graph Analytics

Use built-in graph analytics functions.


In [ ]:
# Get neighbors of a node
neighbors = store.get_neighbors(
    node_id=apple["id"],
    direction="both",
    depth=2
)

print(f"Found {len(neighbors)} neighbors (up to depth 2):")
for neighbor in neighbors:
    print(f"  - {neighbor.get('properties', {}).get('name', 'Unknown')}")


In [ ]:
# Find shortest path (if nodes are connected)
path = store.shortest_path(
    start_node_id=tim_cook["id"],
    end_node_id=cupertino["id"],
    max_depth=5
)

if path:
    print(f"Shortest path length: {path.get('length')}")
    print(f"Nodes in path: {len(path.get('nodes', []))}")
else:
    print("No path found")


## Step 7: Get Graph Statistics

Get statistics about the graph.


In [ ]:
stats = store.get_stats()

print("Graph Statistics:")
print(f"  Node count: {stats.get('node_count', 'N/A')}")
print(f"  Relationship count: {stats.get('relationship_count', 'N/A')}")
print(f"  Label counts: {stats.get('label_counts', {})}")
print(f"  Relationship types: {stats.get('relationship_type_counts', {})}")


## Step 8: Clean Up

Close the connection when done.


In [ ]:
# Close the connection
store.close()
print("Connection closed.")


## Summary

You've learned how to use graph stores:

- **GraphStore**: Unified interface for property graph databases
- **Multiple Backends**: Neo4j, KuzuDB, FalkorDB support
- **Node Operations**: Create, read, update, delete nodes
- **Relationship Operations**: Create and query relationships
- **Cypher Queries**: Execute powerful graph queries
- **Graph Analytics**: Shortest path, neighbors, centrality

### Backend Comparison

| Backend | Best For | Deployment |
|---------|----------|------------|
| **Neo4j** | Enterprise, full features | Server/Cloud |
| **KuzuDB** | Analytics, embedded | Embedded (no server) |
| **FalkorDB** | LLM apps, real-time | Redis-based |

Next: Learn how to visualize graphs in the Visualization notebook.
